In [1]:
import sys
sys.path.insert(0, '/Users/niloufar/Downloads/UNI/winter_2024/EECS 553/final_project/EGC/')
from guided_diffusion.unet import UNetModel


In [2]:
def load_model(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    print("Checkpoint keys:", checkpoint.keys())  # Print all keys in the checkpoint dictionary

    # Attempt to load the state dictionary
    try:
        model.load_state_dict(checkpoint['state_dict'])
    except KeyError:
        print("Key 'state_dict' not found. Available keys:", checkpoint.keys())
        raise
    
    return model

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Import the UNetModel class from the guided_diffusion module
from guided_diffusion.unet import UNetModel

def load_data(batch_size=64):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # Resize images to 32x32
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize data
    ])
    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

    test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader, test_loader

def load_model(checkpoint_path):
    # Initializing unetModel with the same parameters as the original training
    model = UNetModel(
        image_size=32,
        in_channels=3,
        model_channels=384,
        out_channels=10,  # Initially setting this to 10 for CIFAR-10
        num_res_blocks=2,
        attention_resolutions={1, 2, 4},
        dropout=0.1,
        channel_mult=(1, 2, 4, 8)
    )

    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))

    # Load weights with strict=False to allow for ignoring non-matching keys
    model.load_state_dict(checkpoint, strict=False)

    # Manually adjust layers that have size mismatches
    if 'output_blocks.8.1.norm.bias' in checkpoint:
        model.output_blocks[8][1].norm = nn.InstanceNorm2d(768)  # Adjusting normalization layers

    if 'output_blocks.8.1.proj_out.weight' in checkpoint:
        model.output_blocks[8][1].proj_out = nn.Conv2d(768, 768, 1)  # Adjusting projection layer

    # Reinitialize the output layer specifically for CIFAR-10
    model.out = nn.Sequential(
        nn.Conv2d(384, 10, 3, padding=1),
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(10, 10)  # Ensuring it matches CIFAR-10 classes
    )

    return model

checkpoint_path = '/Users/niloufar/Downloads/UNI/winter_2024/EECS 553/final_project/EGC/checkpoints/context_imagenet32_c384_b2_d0_headc64_chmult124_attn32_16_8_convupd_simpleattn_s1k_linear_start_0.00085_end_0.012_bs512_lr1e-4_reweight_alpha_ce0.005_weakaug_training_ema.pt'


    # load_model(checkpoint_path)
    # model.load_state_dict(checkpoint)

    # return model

def main():
    train_loader, test_loader = load_data()
    model = load_model(checkpoint_path)
    
    # Evaluate the model
    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images, torch.tensor([0]*len(images)))  # Assuming 0 is a valid timestep
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()
            print(f'Accuracy: {100 * correct / len(labels)}%')

if __name__ == '__main__':
    main()


Files already downloaded and verified
Files already downloaded and verified
